In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import os
import re
import time

import pandas as pd
import requests
import selenium
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm.notebook import tqdm

<IPython.core.display.Javascript object>

In [3]:
def browser(url):
    option = webdriver.ChromeOptions()
    option.add_argument('headless')
    driver = webdriver.Chrome(options=option)
    driver.get(url)
    return driver


def get_html_full(url):
    driver = browser(url)
    el = driver.find_element_by_xpath(
        "/html/body/of-root/main/of-competition-results-stream/section/of-entity-page-root/div/of-xpa-switch-entity-deprecated[7]/of-simple-match-cards-list-deprecated/div/button"
    )
    el.click()
    time.sleep(5)
    page = driver.page_source
    driver.close()
    return BeautifulSoup(page)


def get_html_full_match(url):
    driver = browser(url)
    el = driver.find_element_by_xpath(
        "/html/body/of-root/main/of-competition-results-stream/section/of-entity-page-root/div/of-xpa-switch-entity-deprecated[7]/of-simple-match-cards-list-deprecated/div/button"
    )
    el.click()
    time.sleep(5)
    page = driver.page_source
    driver.close()
    return BeautifulSoup(page)


def get_html_from_a_match(url):
    driver = browser(url)
    el = driver.find_element_by_xpath(
        "/html/body/of-root/main/of-match-stream-v2/section/of-xpa-layout-match/section[2]/of-xpa-switch-match/of-match-events/div/button"
    )
    el.click()
    time.sleep(2)
    page = driver.page_source
    el = driver.find_element_by_xpath(
        "/html/body/of-root/main/of-match-stream-v2/section/of-xpa-layout-match/section[3]/div[1]/div/of-xpa-switch-match[2]/of-match-lineup/div/nav/ul/li[2]/button"
    )
    el.click()
    time.sleep(2)
    second_lineup = driver.page_source
    driver.close()
    return BeautifulSoup(page), BeautifulSoup(second_lineup)



def format_field(string):
    string = str(string)
    string = string.lower()
    string = string.strip()
    string = string.replace("  ", " ")
    string = string.replace(" ", "_")
    return string


def get_info_from_event(event):
    aux_event = {}
    aux_event["event_team"] = event["class"][-1][-4:]
    aux_event["event_time"] = event.find(
        "div", class_="match-events__item-timeline"
    ).text

    if aux_event["event_time"] == " PK ":
        aux_event["event_time"] = False
        aux_event["event_type"] = "PK"
        aux_event["event_result"] = event.find("img", class_="of-image__img")["alt"]
        aux_event["event_player"] = (
            event.find("div", class_="match-events__text").find("p").text
        )
    else:
        try:
            aux_event["event_type"] = event.find(class_="match-events__icon")[
                "aria-label"
            ]
        except:
            aux_event["event_type"] = event.find("img", class_="of-image__img")["alt"]
        try:
            for i, text in enumerate(
                event.find("div", class_="match-events__text").find_all("p")
            ):
                aux_event["action_player_" + str(i + 1)] = text.text
        except:
            aux_event["action_player_1"] = event.find(
                "p", class_="match-events__text"
            ).text

    return aux_event


def get_info_from_match(page, second_lineup):
    aux_dict = {}

    ## NAMES
    aux_dict["team_name_home"] = page.find(
        "span", class_="match-score-team__name--home"
    ).text
    aux_dict["team_name_away"] = page.find(
        "span", class_="match-score-team__name--away"
    ).text

    ## GOAL
    scores = page.find("p", class_="match-score-scores").find_all("span")
    aux_dict["team_home_score"] = scores[0].text
    aux_dict["team_away_score"] = scores[2].text

    ## STATISTICS
    description = page.find_all("div", class_="match-stats__stat-description")
    for d in description:
        field = format_field(d.find_all("p")[1].text)
        aux_dict[field + "_home"] = d.find_all("p")[0].text
        aux_dict[field + "_away"] = d.find_all("p")[2].text

    ## EVENTS
    events = page.find("ul", class_="match-events__list").find_all(
        "li", class_="match-events__item"
    )
    aux_dict["events_list"] = [get_info_from_event(event) for event in events]

    aux_dict["lineup_home"] = get_lineups(page)
    aux_dict["lineup_away"] = get_lineups(second_lineup)

    return aux_dict


def get_lineups(page):
    lineup = []
    for player in page.find_all("div", class_="match-lineup__player"):
        lineup.append(
            {
                "Player_Name": player.find(
                    "p", class_="match-lineup__player-name"
                ).text,
                "Player_Number": format_field(
                    player.find("span", class_="match-lineup__jersey-number").text
                ),
            }
        )
    return lineup



def get_penaltis(match):
    regex_rule = r"\((\d+)\)"
    aux = match.find_all(
        "span", class_="title-7-bold simple-match-card-team__score"
    )
    aux = [
        re.search(regex_rule, i.text)[0].replace("(", "").replace(")", "") for i in aux
    ]
    return aux


def get_all_matches(html_full):
    aux_dict = []
    for match in html_full.find_all("a", class_="match-card", href=True):
        aux_dict_2 = {}
        aux_dict_2["link"] = "https://onefootball.com" + match["href"]
        aux_dict_2["stage"] = match.find_previous(
            "p", class_="label simple-match-cards-list__title-deprecated"
        ).text
        try:
            aux_dict_2["date"] = match.find("time").text
            aux_dict_2["pens"] = False
            aux_dict_2["pens_home_score"] = False
            aux_dict_2["pens_away_score"] = False
        except:
            aux_dict_2["date"] = False
            aux_dict_2["pens"] = (
                match.find(
                    "span",
                    class_="title-8-medium simple-match-card__info-message simple-match-card__info-message--secondary",
                ).text
                == "(Pens)"
            )
            home, away = get_penaltis(match)
            aux_dict_2["pens_home_score"] = home
            aux_dict_2["pens_away_score"] = away

        aux_dict.append(aux_dict_2)
    return aux_dict


<IPython.core.display.Javascript object>

## Competition

In [4]:
url = "https://onefootball.com/en/competition/champions-league-5/results"  ## Competition link
page = get_html_full(url)
matchs = get_all_matches(page)
matchs[0]

{'link': 'https://onefootball.com/en/match/2203217',
 'stage': ' Final ',
 'date': ' 29.05.2021 ',
 'pens': False,
 'pens_home_score': False,
 'pens_away_score': False}

<IPython.core.display.Javascript object>

In [5]:
%%time

def merge_information(match):
    match_info = get_info_from_match(*get_html_from_a_match(match["link"]))
    return{**match, **match_info}
    
from joblib import Parallel, delayed
aux_list = Parallel(n_jobs=6, verbose=10)(delayed(merge_information)(match) for match in matchs)

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:   12.0s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:   13.9s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:   36.6s
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:   47.1s
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:  1.0min
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:  1.3min
[Parallel(n_jobs=6)]: Done  49 tasks      | elapsed:  1.8min
[Parallel(n_jobs=6)]: Done  60 tasks      | elapsed:  2.1min
[Parallel(n_jobs=6)]: Done  73 tasks      | elapsed:  2.5min
[Parallel(n_jobs=6)]: Done  86 tasks      | elapsed:  2.8min
[Parallel(n_jobs=6)]: Done 101 tasks      | elapsed:  3.4min


Wall time: 4min 12s


[Parallel(n_jobs=6)]: Done 125 out of 125 | elapsed:  4.2min finished


<IPython.core.display.Javascript object>

In [6]:
df = pd.DataFrame(aux_list).drop("link", axis=1)
df.shape

(125, 20)

<IPython.core.display.Javascript object>

## Manual adjustment

In [7]:
df[df.date == False]

,stage,date,pens,pens_home_score,pens_away_score,team_name_home,team_name_away,team_home_score,team_away_score,possession_home,possession_away,total_shots_home,total_shots_away,shots_on_target_home,shots_on_target_away,duels_won_home,duels_won_away,events_list,lineup_home,lineup_away


<IPython.core.display.Javascript object>

In [8]:
df

,stage,date,pens,pens_home_score,pens_away_score,team_name_home,team_name_away,team_home_score,team_away_score,possession_home,possession_away,total_shots_home,total_shots_away,shots_on_target_home,shots_on_target_away,duels_won_home,duels_won_away,events_list,lineup_home,lineup_away
0,Final,29.05.2021,False,False,False,Manchester City,Chelsea,0,1,61%,39%,7,8,1,2,50%,50%,"[{'event_team': 'home', 'event_time': ' 34' ',...","[{'Player_Name': 'Sterling', 'Player_Number': ...","[{'Player_Name': 'Werner', 'Player_Number': '1..."
1,Semi-finals second leg,05.05.2021,False,False,False,Chelsea,Real Madrid,2,0,32%,68%,15,7,5,5,52%,49%,"[{'event_team': 'home', 'event_time': ' 14' ',...","[{'Player_Name': 'Werner', 'Player_Number': '1...","[{'Player_Name': 'Hazard', 'Player_Number': '7..."
2,Semi-finals second leg,04.05.2021,False,False,False,Manchester City,PSG,2,0,44%,56%,12,14,5,0,53%,47%,"[{'event_team': 'home', 'event_time': ' 11' ',...","[{'Player_Name': 'Silva', 'Player_Number': '20...","[{'Player_Name': 'Neymar', 'Player_Number': '1..."
3,Semi-finals first leg,28.04.2021,False,False,False,PSG,Manchester City,1,2,40%,60%,10,11,4,6,47%,53%,"[{'event_team': 'home', 'event_time': ' 15' ',...","[{'Player_Name': 'Mbappe', 'Player_Number': '7...","[{'Player_Name': 'Foden', 'Player_Number': '47..."
4,Semi-finals first leg,27.04.2021,False,False,False,Real Madrid,Chelsea,1,1,51%,49%,9,11,1,5,49%,52%,"[{'event_team': 'away', 'event_time': ' 14' ',...","[{'Player_Name': 'Junior', 'Player_Number': '2...","[{'Player_Name': 'Werner', 'Player_Number': '1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,Group stage: Matchday 1,20.10.2020,False,False,False,Rennes,Krasnodar,1,1,63%,37%,20,10,7,3,49%,51%,"[{'event_team': 'away', 'event_time': ' 14' ',...","[{'Player_Name': 'Terrier', 'Player_Number': '...","[{'Player_Name': 'Berg', 'Player_Number': '33'..."
121,Group stage: Matchday 1,20.10.2020,False,False,False,Barcelona,Ferencváros,5,1,62%,38%,22,8,11,2,62%,38%,"[{'event_team': 'home', 'event_time': ' 27' ',...","[{'Player_Name': 'Messi', 'Player_Number': '10...","[{'Player_Name': 'Chol Nguen', 'Player_Number'..."
122,Group stage: Matchday 1,20.10.2020,False,False,False,Lazio,Borussia Dortmund,3,1,35%,65%,11,14,4,7,47%,53%,"[{'event_team': 'home', 'event_time': ' 6' ', ...","[{'Player_Name': 'Immobile', 'Player_Number': ...","[{'Player_Name': 'Haaland', 'Player_Number': '..."
123,Group stage: Matchday 1,20.10.2020,False,False,False,Zenit,Club Brugge,1,2,52%,48%,14,13,3,5,45%,55%,"[{'event_team': 'away', 'event_time': ' 12' ',...","[{'Player_Name': 'Dzyuba', 'Player_Number': '2...","[{'Player_Name': 'De Ketelaere', 'Player_Numbe..."


<IPython.core.display.Javascript object>

In [9]:
df.to_csv("champions_league_2020-2021_results.csv", index=False)

<IPython.core.display.Javascript object>